In [136]:
import pandas as pd
from pymorphy2 import MorphAnalyzer
from russian_tagsets import converters
from nltk.corpus import stopwords



In [107]:
import numpy as np
import seaborn as sns
from multiprocessing import Pool

num_partitions = 10 #number of partitions to split dataframe
num_cores = 4 #number of cores on your machine

def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [108]:
stopwords_russian = stopwords.words('russian')

In [109]:
df_train = pd.read_csv('train_data.csv')

In [137]:
df_test = pd.read_csv('test_data.csv')

In [110]:
df_train = df_train.set_index('ID')

In [111]:
#imbalanced of classes 
df_train[df_train['Revisit'] == 0].size / df_train.size, df_train[df_train['Revisit'] == 1].size / df_train.size

(0.17136135308704775, 0.8286386469129522)

In [112]:
df_train[df_train['аллергия'].notnull()].size

368172

In [113]:
df_train.size

2547774

In [126]:
morph = MorphAnalyzer()
to_ud = converters.converter('opencorpora-int', 'ud14')

def convert_from_opencorpora_tag(tag, text):
    ud_tag = to_ud(str(tag), text)
    pos = ud_tag.split()[0]
    gram = ud_tag.split()[1]
    return pos, gram

def norm_sentence(sentence):
    if not sentence or not isinstance(sentence, str):
        return None
    words = [w for w in sentence.split(' ') if len(w) > 1 and w not in stopwords_russian]
    normal = []
    for i, word in enumerate(words):
        form = morph.parse(word)[0]
        pos, gram = convert_from_opencorpora_tag(form.tag, word)
        normal.append(form.normal_form)
    return normal

In [115]:
norm_sentence(df_train['Диагноз'][0])

['острый', 'ларингофарингит']

In [123]:
def morph_sentences(df):
    df['Диагноз'] = df['Диагноз'].apply(lambda x: norm_sentence(x))
    return df

def morph_sentences_resid(df):
    df['Общее состояние'] = df['Общее состояние'].apply(lambda x: norm_sentence(x))
    df['аллергия'] = df['аллергия'].apply(lambda x: norm_sentence(x))
    df['Анамнез заболевания'] = df['Анамнез заболевания'].apply(lambda x: norm_sentence(x))
    df['Внешний осмотр'] = df['Внешний осмотр'].apply(lambda x: norm_sentence(x))
    return df

In [117]:
df_train = parallelize_dataframe(df_train, morph_sentences)

In [125]:
df_train

,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit
ID,,,,,,,,,
0,J06.0,"[острый, ларингофарингит]",29,2,NaN,NaN,NaN,На жевательной поверхности 2.6- кариозная поло...,1
1,N76.1,"[подострый, хронический, вагинит]",45,2,NaN,NaN,считает себя больной на протяжении многих лет....,NaN,1
2,L23.9,"[аллергический, контактный, дерматит,, причина...",63,2,NaN,NaN,"Болен около 2-х дней, когда появились описанны...",NaN,1
3,N48.1,[баланопостить],43,2,NaN,не отягощен,NaN,NaN,1
4,Z00.0,"[общий, медицинский, осмотр]",29,1,NaN,NaN,провела полное обследование для оперативного л...,NaN,1
5,Z01,"[другой, специальный, осмотр, обследование, ли...",50,2,удовлетворительное.,не отягощен,NaN,NaN,1
6,M23.2,"[поражение, мениск, результат, старое, разрыв,...",42,2,NaN,NaN,"проводит лечение, прибыла для коррекции назнач...",NaN,1
7,E05.0,"[тиреотоксикоз, диффузный, зоб]",85,1,NaN,NaN,Вышеописанные жалобы беспокоят с 03.12.17. Об...,NaN,0
8,N77.1*,"[вагинит,, вульвит, вульвовагинит, инфекционны...",28,2,Общее самочувствие удовлетворительное. Кожные ...,NaN,NaN,NaN,1


In [127]:
%%time
df_train = parallelize_dataframe(df_train, morph_sentences_resid)

CPU times: user 3.41 s, sys: 1.79 s, total: 5.2 s
Wall time: 5min 55s


In [133]:
df_train

,Код_диагноза,Диагноз,Возраст,Пол,Общее состояние,аллергия,Анамнез заболевания,Внешний осмотр,Revisit
ID,,,,,,,,,
0,J06.0,"[острый, ларингофарингит]",29,2,None,None,None,"[на, жевательный, поверхность, 2.6-, кариозный...",1
1,N76.1,"[подострый, хронический, вагинит]",45,2,None,None,"[считать, больной, протяжение, многий, лет., п...",None,1
2,L23.9,"[аллергический, контактный, дерматит,, причина...",63,2,None,None,"[больной, около, 2-х, дней,, появиться, описат...",None,1
3,N48.1,[баланопостить],43,2,None,[отяготить],None,None,1
4,Z00.0,"[общий, медицинский, осмотр]",29,1,None,None,"[провести, полный, обследование, оперативный, ...",None,1
5,Z01,"[другой, специальный, осмотр, обследование, ли...",50,2,[удовлетворительное.],[отяготить],None,None,1
6,M23.2,"[поражение, мениск, результат, старое, разрыв,...",42,2,None,None,"[проводить, лечение,, прибыть, коррекция, назн...",None,1
7,E05.0,"[тиреотоксикоз, диффузный, зоб]",85,1,None,None,"[вышеописанный, жалоба, беспокоить, 03.12.17.,...",None,0
8,N77.1*,"[вагинит,, вульвит, вульвовагинит, инфекционны...",28,2,"[общий, самочувствие, удовлетворительное., кож...",None,None,None,1


In [134]:
df_train.to_csv('df_train_morph.csv')

In [138]:
%%time
df_test = parallelize_dataframe(df_test, morph_sentences)
df_test = parallelize_dataframe(df_test, morph_sentences_resid)

CPU times: user 1.58 s, sys: 658 ms, total: 2.24 s
Wall time: 2min 21s


In [139]:
df_test.to_csv('df_test_morph.csv')